In [1]:
import numpy as np
import pandas as pd
import nltk
import string

In [2]:
import kagglehub
path = kagglehub.dataset_download("emineyetm/fake-news-detection-datasets")
print("Path to dataset files:", path)

100%|██████████| 41.0M/41.0M [00:20<00:00, 2.09MB/s]

Extracting files...


Path to dataset files: C:\Users\KIIT0001\.cache\kagglehub\datasets\emineyetm\fake-news-detection-datasets\versions\1


In [3]:
import os
print(os.listdir(path))

['News _dataset']


In [4]:
import os
print(os.listdir(path))
print(os.listdir(os.path.join(path, "News _dataset")))

['News _dataset']
['Fake.csv', 'True.csv']


In [5]:
import pandas as pd
import os
folder = os.path.join(path, "News _dataset")

fake_df = pd.read_csv(os.path.join(folder, "Fake.csv"))
true_df = pd.read_csv(os.path.join(folder, "True.csv"))
print(fake_df.head())

                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  


In [6]:
fake_df.shape

(23481, 4)

In [7]:
fake_df["label"] = 0
true_df["label"] = 1

df = pd.concat([fake_df, true_df], ignore_index=True)
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

In [8]:
df.shape

(44898, 5)

In [9]:
df = df.sample(n=20000, random_state=42).reset_index(drop=True)
print(df.shape)

(20000, 5)


In [10]:
df.head()

,title,text,subject,date,label
0,It’s Really Happening: Trump Adviser Lays Out...,"Well, that didn t take long. In the short time...",News,"November 16, 2016",0
1,Republican attempt to deflect Trump-Russia pro...,(Reuters) - Republican lawmaker Devin Nunes’ i...,politicsNews,"September 11, 2017",1
2,Trump says churches should get FEMA funds for ...,WASHINGTON (Reuters) - President Donald Trump ...,politicsNews,"September 9, 2017",1
3,Trump Loves To Say The New York Times Is ‘Fai...,Print journalism and longstanding papers have ...,News,"August 7, 2017",0
4,House Speaker Ryan briefed Trump on healthcare...,WASHINGTON (Reuters) - President Donald Trump ...,politicsNews,"March 24, 2017",1


In [11]:
df = df.drop(columns=["subject", "date"])
print(df.columns)

Index(['title', 'text', 'label'], dtype='object')


In [12]:
import re
import nltk
import string
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

nltk.download("punkt")
nltk.download("stopwords")
nltk.download("punkt_tab")
stop = set(stopwords.words("english"))
ps = PorterStemmer()

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [13]:
import re

def transform(text):
    text = re.sub(r'<user>', '', str(text))
    text = text.lower()
    tokens = nltk.word_tokenize(text)   # <- uses punkt
    tokens = [word for word in tokens if word.isalnum()]
    tokens = [word for word in tokens if word not in stop]
    tokens = [ps.stem(word) for word in tokens]
    return " ".join(tokens)

In [14]:
print(df.columns.tolist())

['title', 'text', 'label']


In [15]:
df['text'] = df['text'].apply(transform)
df['title'] = df['title'].apply(transform)

In [16]:
df.head()

,title,text,label
0,realli happen trump advis lay plan nation musl...,well take long short time sinc american elect ...,0
1,republican attempt deflect probe could backfir...,reuter republican lawmak devin nune investig w...,1
2,trump say church get fema fund hurrican relief,washington reuter presid donald trump said fri...,1
3,trump love say new york time fail effortlessli...,print journal longstand paper struggl sinc adv...,0
4,hous speaker ryan brief trump healthcar bill v...,washington reuter presid donald trump brief fr...,1


In [17]:
df = df.drop_duplicates(subset=['title', 'text'])

In [18]:
nltk.download('wordnet')
wn_lemmatizer = nltk.WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\KIIT0001\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
df["content"] = df["title"].fillna("") + " " + df["text"].fillna("")


In [20]:
from sklearn.model_selection import train_test_split

X_train_text, X_test_text, y_train, y_test = train_test_split(
    df["content"], df["label"],
    test_size=0.2,
    stratify=df["label"],
    random_state=42,
    shuffle=True
)

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer(max_features=5000, ngram_range=(1,2))
X_train = vectorizer.fit_transform(X_train_text)
X_test = vectorizer.transform(X_test_text)

In [26]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(
    n_estimators=50,
    max_depth=3,
    random_state=42,
    n_jobs=-1
)

In [27]:
from sklearn.metrics import classification_report, confusion_matrix

In [28]:
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.97      0.93      0.95      1862
           1       0.93      0.98      0.95      1894

    accuracy                           0.95      3756
   macro avg       0.95      0.95      0.95      3756
weighted avg       0.95      0.95      0.95      3756

[[1730  132]
 [  47 1847]]


In [29]:
test_text = "Breaking: Scientists confirm that this new virus strain is completely fake news."

In [30]:
# Apply the same transform function
processed_text = transform(test_text)

In [31]:
X_test_example = vectorizer.transform([processed_text])

In [32]:
prediction = rf.predict(X_test_example)[0]

if prediction == 0:
    print("Prediction: Fake News")
else:
    print("Prediction: Real News")

Prediction: Fake News


In [44]:
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(X_train, y_train)
y_pred = mnb.predict(X_test)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.94      0.94      1862
           1       0.94      0.94      0.94      1894

    accuracy                           0.94      3756
   macro avg       0.94      0.94      0.94      3756
weighted avg       0.94      0.94      0.94      3756

[[1747  115]
 [ 121 1773]]


In [45]:
# Save model and vectorizer
import pickle
with open("text_model.pkl", "wb") as f:
    pickle.dump(mnb, f)

with open("vectorizer.pkl", "wb") as f:
    pickle.dump(vectorizer, f)

print("Model and vectorizer saved!")

Model and vectorizer saved!
